In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='

p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]
    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./{FileName}')        # 동일 폴더에 파일이 저장될 거임
    return df

In [ ]:
past=4
pBus=Bus(p7000)
for x in range(720): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000_tue.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0]]
1 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0]]
2 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0]]
3 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0]]
4 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1]]
5 번째 시도:  [[

16 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1], ['2022-08-02 13:31:16.003', [2, 14, 26, 33, 48, 52, 54, 63], 0], ['2022-08-02 13:32:16.158', [3, 14, 26, 34, 49, 52, 55, 63], 0], ['2022-08-02 13:33:16.330', [3, 15, 26, 35, 50, 52, 55, 64], 0], ['2022-08-02 13:34:23.486', [3, 16, 26, 36, 50, 53, 55, 65], 0], ['2022-08-02 13:35:23.642', [4, 16, 27, 37, 51, 53, 57, 65], 0], ['2022-08-02 13:36:23.782', [5, 17, 27, 37, 51, 54, 58, 66], 0], ['2022-08-02 13:37:23.954', [7, 18, 28, 38, 51, 55, 59, 68], 0], ['2022-08-02 13:38:25.516', [7, 18, 28, 38, 52, 55, 61, 68], 0], ['2022-08-02 13:39:25.702', [8, 18, 29, 38, 52, 56, 62, 69], 0], ['2022-08-02 13:40:26.514', [8, 19, 29, 39, 52, 56, 62, 69], 0], ['2022-08-02 13:41:26.669

23 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1], ['2022-08-02 13:31:16.003', [2, 14, 26, 33, 48, 52, 54, 63], 0], ['2022-08-02 13:32:16.158', [3, 14, 26, 34, 49, 52, 55, 63], 0], ['2022-08-02 13:33:16.330', [3, 15, 26, 35, 50, 52, 55, 64], 0], ['2022-08-02 13:34:23.486', [3, 16, 26, 36, 50, 53, 55, 65], 0], ['2022-08-02 13:35:23.642', [4, 16, 27, 37, 51, 53, 57, 65], 0], ['2022-08-02 13:36:23.782', [5, 17, 27, 37, 51, 54, 58, 66], 0], ['2022-08-02 13:37:23.954', [7, 18, 28, 38, 51, 55, 59, 68], 0], ['2022-08-02 13:38:25.516', [7, 18, 28, 38, 52, 55, 61, 68], 0], ['2022-08-02 13:39:25.702', [8, 18, 29, 38, 52, 56, 62, 69], 0], ['2022-08-02 13:40:26.514', [8, 19, 29, 39, 52, 56, 62, 69], 0], ['2022-08-02 13:41:26.669

28 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1], ['2022-08-02 13:31:16.003', [2, 14, 26, 33, 48, 52, 54, 63], 0], ['2022-08-02 13:32:16.158', [3, 14, 26, 34, 49, 52, 55, 63], 0], ['2022-08-02 13:33:16.330', [3, 15, 26, 35, 50, 52, 55, 64], 0], ['2022-08-02 13:34:23.486', [3, 16, 26, 36, 50, 53, 55, 65], 0], ['2022-08-02 13:35:23.642', [4, 16, 27, 37, 51, 53, 57, 65], 0], ['2022-08-02 13:36:23.782', [5, 17, 27, 37, 51, 54, 58, 66], 0], ['2022-08-02 13:37:23.954', [7, 18, 28, 38, 51, 55, 59, 68], 0], ['2022-08-02 13:38:25.516', [7, 18, 28, 38, 52, 55, 61, 68], 0], ['2022-08-02 13:39:25.702', [8, 18, 29, 38, 52, 56, 62, 69], 0], ['2022-08-02 13:40:26.514', [8, 19, 29, 39, 52, 56, 62, 69], 0], ['2022-08-02 13:41:26.669

33 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1], ['2022-08-02 13:31:16.003', [2, 14, 26, 33, 48, 52, 54, 63], 0], ['2022-08-02 13:32:16.158', [3, 14, 26, 34, 49, 52, 55, 63], 0], ['2022-08-02 13:33:16.330', [3, 15, 26, 35, 50, 52, 55, 64], 0], ['2022-08-02 13:34:23.486', [3, 16, 26, 36, 50, 53, 55, 65], 0], ['2022-08-02 13:35:23.642', [4, 16, 27, 37, 51, 53, 57, 65], 0], ['2022-08-02 13:36:23.782', [5, 17, 27, 37, 51, 54, 58, 66], 0], ['2022-08-02 13:37:23.954', [7, 18, 28, 38, 51, 55, 59, 68], 0], ['2022-08-02 13:38:25.516', [7, 18, 28, 38, 52, 55, 61, 68], 0], ['2022-08-02 13:39:25.702', [8, 18, 29, 38, 52, 56, 62, 69], 0], ['2022-08-02 13:40:26.514', [8, 19, 29, 39, 52, 56, 62, 69], 0], ['2022-08-02 13:41:26.669

...

165 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1], ['2022-08-02 13:31:16.003', [2, 14, 26, 33, 48, 52, 54, 63], 0], ['2022-08-02 13:32:16.158', [3, 14, 26, 34, 49, 52, 55, 63], 0], ['2022-08-02 13:33:16.330', [3, 15, 26, 35, 50, 52, 55, 64], 0], ['2022-08-02 13:34:23.486', [3, 16, 26, 36, 50, 53, 55, 65], 0], ['2022-08-02 13:35:23.642', [4, 16, 27, 37, 51, 53, 57, 65], 0], ['2022-08-02 13:36:23.782', [5, 17, 27, 37, 51, 54, 58, 66], 0], ['2022-08-02 13:37:23.954', [7, 18, 28, 38, 51, 55, 59, 68], 0], ['2022-08-02 13:38:25.516', [7, 18, 28, 38, 52, 55, 61, 68], 0], ['2022-08-02 13:39:25.702', [8, 18, 29, 38, 52, 56, 62, 69], 0], ['2022-08-02 13:40:26.514', [8, 19, 29, 39, 52, 56, 62, 69], 0], ['2022-08-02 13:41:26.66

166 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1], ['2022-08-02 13:31:16.003', [2, 14, 26, 33, 48, 52, 54, 63], 0], ['2022-08-02 13:32:16.158', [3, 14, 26, 34, 49, 52, 55, 63], 0], ['2022-08-02 13:33:16.330', [3, 15, 26, 35, 50, 52, 55, 64], 0], ['2022-08-02 13:34:23.486', [3, 16, 26, 36, 50, 53, 55, 65], 0], ['2022-08-02 13:35:23.642', [4, 16, 27, 37, 51, 53, 57, 65], 0], ['2022-08-02 13:36:23.782', [5, 17, 27, 37, 51, 54, 58, 66], 0], ['2022-08-02 13:37:23.954', [7, 18, 28, 38, 51, 55, 59, 68], 0], ['2022-08-02 13:38:25.516', [7, 18, 28, 38, 52, 55, 61, 68], 0], ['2022-08-02 13:39:25.702', [8, 18, 29, 38, 52, 56, 62, 69], 0], ['2022-08-02 13:40:26.514', [8, 19, 29, 39, 52, 56, 62, 69], 0], ['2022-08-02 13:41:26.66

167 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1], ['2022-08-02 13:31:16.003', [2, 14, 26, 33, 48, 52, 54, 63], 0], ['2022-08-02 13:32:16.158', [3, 14, 26, 34, 49, 52, 55, 63], 0], ['2022-08-02 13:33:16.330', [3, 15, 26, 35, 50, 52, 55, 64], 0], ['2022-08-02 13:34:23.486', [3, 16, 26, 36, 50, 53, 55, 65], 0], ['2022-08-02 13:35:23.642', [4, 16, 27, 37, 51, 53, 57, 65], 0], ['2022-08-02 13:36:23.782', [5, 17, 27, 37, 51, 54, 58, 66], 0], ['2022-08-02 13:37:23.954', [7, 18, 28, 38, 51, 55, 59, 68], 0], ['2022-08-02 13:38:25.516', [7, 18, 28, 38, 52, 55, 61, 68], 0], ['2022-08-02 13:39:25.702', [8, 18, 29, 38, 52, 56, 62, 69], 0], ['2022-08-02 13:40:26.514', [8, 19, 29, 39, 52, 56, 62, 69], 0], ['2022-08-02 13:41:26.66

168 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1], ['2022-08-02 13:31:16.003', [2, 14, 26, 33, 48, 52, 54, 63], 0], ['2022-08-02 13:32:16.158', [3, 14, 26, 34, 49, 52, 55, 63], 0], ['2022-08-02 13:33:16.330', [3, 15, 26, 35, 50, 52, 55, 64], 0], ['2022-08-02 13:34:23.486', [3, 16, 26, 36, 50, 53, 55, 65], 0], ['2022-08-02 13:35:23.642', [4, 16, 27, 37, 51, 53, 57, 65], 0], ['2022-08-02 13:36:23.782', [5, 17, 27, 37, 51, 54, 58, 66], 0], ['2022-08-02 13:37:23.954', [7, 18, 28, 38, 51, 55, 59, 68], 0], ['2022-08-02 13:38:25.516', [7, 18, 28, 38, 52, 55, 61, 68], 0], ['2022-08-02 13:39:25.702', [8, 18, 29, 38, 52, 56, 62, 69], 0], ['2022-08-02 13:40:26.514', [8, 19, 29, 39, 52, 56, 62, 69], 0], ['2022-08-02 13:41:26.66

169 번째 시도:  [['2022-08-02 13:26:12.850', [11, 25, 31, 46, 50, 52, 60], 0], ['2022-08-02 13:27:13.021', [12, 25, 32, 46, 51, 52, 60], 0], ['2022-08-02 13:28:13.146', [13, 25, 32, 47, 51, 53, 61], 0], ['2022-08-02 13:29:13.286', [13, 25, 33, 47, 51, 53, 63], 0], ['2022-08-02 13:30:13.425', [1, 14, 26, 33, 48, 51, 53, 63], 1], ['2022-08-02 13:31:16.003', [2, 14, 26, 33, 48, 52, 54, 63], 0], ['2022-08-02 13:32:16.158', [3, 14, 26, 34, 49, 52, 55, 63], 0], ['2022-08-02 13:33:16.330', [3, 15, 26, 35, 50, 52, 55, 64], 0], ['2022-08-02 13:34:23.486', [3, 16, 26, 36, 50, 53, 55, 65], 0], ['2022-08-02 13:35:23.642', [4, 16, 27, 37, 51, 53, 57, 65], 0], ['2022-08-02 13:36:23.782', [5, 17, 27, 37, 51, 54, 58, 66], 0], ['2022-08-02 13:37:23.954', [7, 18, 28, 38, 51, 55, 59, 68], 0], ['2022-08-02 13:38:25.516', [7, 18, 28, 38, 52, 55, 61, 68], 0], ['2022-08-02 13:39:25.702', [8, 18, 29, 38, 52, 56, 62, 69], 0], ['2022-08-02 13:40:26.514', [8, 19, 29, 39, 52, 56, 62, 69], 0], ['2022-08-02 13:41:26.66